In [11]:
##需要重新安装import torch_utils as tu

import pandas as pd
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import os
import cv2
import timm

import albumentations
from albumentations import pytorch as AT

#below are all from https://github.com/seefun/TorchUtils, thanks seefun to provide such useful tools
#import torch_utils as tu
import torch.utils as tu
FOLD = 5

In [12]:
#导入数据集
#其中transform已删除

train_data=torchvision.datasets .ImageFolder (root='./fruits-360-original-size/Training' )
val_data=torchvision.datasets .ImageFolder (root='./fruits-360-original-size/Validation' )
test_ds=torchvision.datasets .ImageFolder (root='./fruits-360-original-size/Test' )
print(train_data.classes)  #输出train_data里面的文件夹名称

#制作dataloader、即可迭代的数据装载器
batchsize=500   #每次抓取的数据数量
train_loader=DataLoader(train_data,batch_size=batchsize,shuffle=True,num_workers=1)
val_loader=DataLoader(val_data,batch_size=batchsize,shuffle=True,num_workers=1)
test_loader = DataLoader(test_ds, batch_size=batchsize, shuffle=False, num_workers=2)
#如何获得读入的图片总数？

['apple_6', 'apple_braeburn_1', 'apple_crimson_snow_1', 'apple_golden_1', 'apple_golden_2', 'apple_golden_3', 'apple_granny_smith_1', 'apple_hit_1', 'apple_pink_lady_1', 'apple_red_1', 'apple_red_2', 'apple_red_3', 'apple_red_delicios_1', 'apple_red_yellow_1', 'apple_rotten_1', 'cabbage_white_1', 'carrot_1', 'cucumber_1', 'cucumber_3', 'eggplant_violet_1', 'pear_1', 'pear_3', 'zucchini_1', 'zucchini_dark_1']


In [13]:
model = timm.create_model('resnet50d', pretrained=True)

In [14]:
model.cuda()
device = 'cuda'

In [17]:
def get_learner(lr, nb, epochs, model_name='resnet50d', MIXUP=0.1):
    #？输入项nb
  #mixup_fn = tu.Mixup(prob=MIXUP, switch_prob=0.0, onehot=True, label_smoothing=0.05, num_classes=len(train_loader))
  model = timm.create_model(model_name, pretrained=True)
    #这一步设置了什么？具体内容  model.cuda  model.train()等用法
  model.fc = nn.Linear(model.fc.in_features, len(train_loader))
  nn.init.xavier_uniform_(model.fc.weight)
  model.cuda()
    #？

  params_1x = [param for name, param in model.named_parameters()
              if name not in ["fc.weight", "fc.bias"]]

  optimizer = torch.optim.AdamW([{'params': params_1x},
                                    {'params': model.fc.parameters(),
                                      'lr': lr*10}],
                                  lr=lr, weight_decay=2e-4)
#loss_fn  损失函数 (Loss Function)
  loss_fn = nn.CrossEntropyLoss()
  loss_fn_test = F.cross_entropy
#用交叉熵区别两个概率的区别

  lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs*nb, eta_min=lr/20)
    #？
  return model, optimizer, loss_fn, loss_fn_test, lr_scheduler

In [21]:
import ttach as tta
device = 'cuda'
save_dir = './gdrive/MyDrive/kaggle_leave_classification/'
#map from idx to string？？
####去除了labelmap部分
EPOCHS = 10
#mixup是什么？用在哪里？
MIXUP = 0.1

 
scaler = torch.cuda.amp.GradScaler() # for AMP training Automatic Mixed Precision,
#AMP可以在神经网络推理过程中，针对不同的层，采用不同的数据精度进行计算，从而实现节省显存和加快速度的目的
#有了epochs为什么还要fold循环5次？？？因为做5则交叉验证，省数据
for fold in range(FOLD):
  print(f'Start Fold{fold}...')

  train_dl=train_loader
  val_dl=val_loader
  model, optimizer, loss_fn, loss_fn_test, lr_scheduler = get_learner(3e-4, len(train_dl), EPOCHS, model_name='resnet50d', MIXUP=MIXUP)
  model_name = f'5fold_test_fold{fold}'
  train_losses = []
  val_losses = []
  train_accus = []
  val_accus = []
  best_accu = 0
  best_loss = float('inf')
  lrs = []
  for epoch in range(EPOCHS):
          val_accu = 0
          train_accu = 0
          train_losses_tmp = []
          #Train
          model.train()
          t_inf = 0
          #for x, y in train_dl:
              #if MIXUP:
               # x, y = mixup_fn(x, y)
             # x, y = x.to(device), y.to(device)
              #Forward
              #torch.cuda.amp.autocast短训练时间，降低存储需求，作用在哪一类代码上？
             # with torch.cuda.amp.autocast():
                #pred = model(x)
               # loss = loss_fn(pred, y)
                #mixup？
                
              #Backward
              #loss.backward()
              #optimizer.step()
              #scaler.scale(loss).backward()
              #scaler.step(optimizer)
             # scaler.update()
             # lr_scheduler.step()
              #optimizer.zero_grad()
              #Statistics
              #lrs.append(optimizer.param_groups[0]['lr']) #group 0,1,2 share the learning rate
             # train_losses_tmp.append(loss.data.item())
              #pred_labels = torch.argmax(pred.data, dim=1)
              #y_labels = torch.argmax(y.data, dim=1) if MIXUP else y.data
              #train_accu += (pred_labels==y_labels).float().sum()
          #t_inf /= len(train_dl)
          #train_losses.append(np.mean(np.array(train_losses_tmp)))
         ###### train_accu /= n_train
          #train_accus.append(train_accu.data.item())

            
            #loss是训练集的损失值，val_loss是测试集的损失值
          #Validation
          val_losses_tmp = []
          model.eval()
          with torch.no_grad():
            for x, y in val_dl:
                x, y = x.to(device), y.to(device)
                logit = model(x)
                val_loss = loss_fn_test(logit, y) 
                val_losses_tmp.append(val_loss.data.item())
                pred = torch.argmax(logit.data, dim=1)
                val_accu += (pred==y.data).float().sum()
          val_loss = np.mean(np.array(val_losses_tmp))
          val_losses.append(val_loss)
          #######val_accu /= n_val
          val_accus.append(val_accu.data.item())
          print('fold', fold, 'epoch', epoch, 'train_loss', train_losses[epoch], 'val_loss', val_losses[epoch], 'val_accu', val_accu, 'train_accu', train_accu, 'lr[0]', lrs[-1])
          if save_dir is not None:
              if val_accu == best_accu:
                  if val_loss < best_loss: #never satisfied
                      checkpoint = {"model": model.state_dict()}
                      torch.save(checkpoint, os.path.join(save_dir,f'{model_name}_best.pth'))
                      print(f'Stored a new best model in {save_dir}')
                      best_loss = val_loss
              elif val_accu > best_accu:
                  checkpoint = {"model": model.state_dict()}
                  torch.save(checkpoint, os.path.join(save_dir,f'{model_name}_best.pth'))
                  print(f'Stored a new best model in {save_dir}')
                  best_accu = val_accu
              '''
              if epoch == EPOCHS - 1:
                  checkpoint = {"model": model.state_dict()}
                  torch.save(checkpoint, os.path.join(save_dir,f'{model_name}_last.pth'))
                  print(f'Stored the last model in {save_dir}')
              '''
  #test time
  model.eval()
  tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.flip_transform(),  merge_mode='mean')
  tta_model.eval()
  res = []
  for x in test_dl:
      x = x.to(device)
      logit = tta_model(x)
      pred = torch.argmax(logit.data, dim=1).cpu().numpy()
      for i in range(len(pred)):
        res.append(labelmap_inverse[pred[i]])
        #append在列表末尾添加新的对象

  #test_csv.insert(1, 'label', res)
  #test_csv.to_csv(f'submission_e50{model_name}_fold{fold}.csv'), index=False)
    #如何保存，以csv的形式保存？？？
  #print('test cvs is saved')
#输出分类结果？？

Start Fold0...


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "D:\Ananconda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "D:\Ananconda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "D:\Ananconda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "D:\Ananconda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "D:\Ananconda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\collate.py", line 86, in default_collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays

In [ ]:
%matplotlib inline
plt.plot(train_accus)
plt.plot(val_accus)
plt.plot(train_losses)
plt.plot(val_losses)
plt.ylim(0, 1.3)
plt.legend(['train_accus', 'val_accus', 'train_losses', 'val_losses'])
#plt.show()
plt.title('Learning Curve')
plt.xlabel('epochs')

plt.savefig(f'{model_name}_acc98d34.png')